In [10]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

### Generate Dataset

In [14]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == ():
            return None
        

        cropped_face = None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Camera not found")
        return
    
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"./data/y.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

            cv2.imshow("Cropped Face", face)
            if cv2.waitKey(1) == 13 or int(img_id) == 100:
                break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting Samples Complete!!!")


In [13]:
generate_dataset()

### Create Label

In [15]:
def my_label(image_name):
    name = image_name.split(".")[-3]
    # If you have two people in the dataset
    if name == "x":
        return np.array([0,1])
    elif name == "y":
        return np.array([1,0])

### Create Data

In [16]:
def my_data():
    data = []
    for img in tqdm(os.listdir("./data")):
        path = os.path.join("./data", img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)
    return data

In [18]:
data = my_data()

100%|██████████| 100/100 [00:00<00:00, 3346.96it/s]


In [19]:
train = data[:80]
test = data[80:]
x_train = np.array([i[0] for i in train]).reshape(-1,50,50,1)
print(x_train.shape)
y_train = np.array([i[1] for i in train])
x_test = np.array([i[0] for i in test]).reshape(-1,50,50,1)
print(x_test.shape)
y_test = np.array([i[1] for i in test])

(80, 50, 50, 1)
(20, 50, 50, 1)


### Creating the Model